Notebook commun pour le projet


Groupe Emission:

In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import numba as nb
import cython as cy
import scipy
import const


Retour point par point sur les différentes fonctions définies et vérification de leur cohérence:


Définition du paramètre $\Gamma$ avec les fonctions de Bessel et $\xi$

On a d'après l'équation (12):
$$ \Gamma = \frac{K_{3}(\xi)}{K_{2}(\xi)} - \frac{1}{\xi}    $$
Sachant que par analyse dimensionnelle $\xi$ est sans unité

In [6]:
#Définition des fonctions de Bessel de première espèce (avec une limite finie en 0)
def K1(x) :
    return scipy.special.kn(1, x)

def K2(x) :
    return scipy.special.kn(2, x)

def K3(x) :
    return scipy.special.kn(3, x)



In [7]:
def xi():
    def eq(x):
        return 4*K2(x)/(x*(3*K3(x) + K1(x) -4*K2(x))) - x*(const.k_b**2)/(const.m_p**2)*(const.celer**4)*((K3(x)/K2(x)) - (1/x) - 1)
    guess = 0.0
    sol = scipy.optimize.root(eq, guess) 
    return sol

On définit maintenant la fonction $\hat{\gamma}'$:

In [2]:
def gamma_chapeau_p():
    return 1 + 4*K2(xi())/(xi()*(3*K3(xi()) + K1(xi()) -4*K2(xi())))

def gamma_maj(gamma_sh):
    def eq5(x):
        return (x + 1)*(1 + gamma_chapeau_p()*(x - 1))/(2 + gamma_chapeau_p()*(2 - gamma_chapeau_p())*(x - 1)) - gamma_sh()**2
    guess = 0.0
    sol = scipy.optimize.root(eq5,guess) 
    return sol




Définition de la fonction $\frac{e_{i}}{n '}$ :

In [11]:
def e_i_p(gamma_maj, gamma_sh):
    return const.m_p*((const.celer)**2)(gamma_maj(gamma_sh) - 1)



In [8]:
epsi_b = 4.12*(1/np.power(10,5))
def gamma_prime_c(n0,epsilon_B, gamma_sh,t):
    return (3 * const.m_e * const.celer* gamma_maj(gamma_sh)) / (4 * const.sigma_T * epsilon_B *e_i_p(gamma_maj, gamma_sh) * t*n0)

def gamma_prime_m(p, gamma_maj, gamma_sh):
    arg = ((p - 2)*const.m_p/(p - 1)*const.m_e)*const.epsi_e*(gamma_maj(gamma_sh) - 1)
    return np.max(1, arg)




Définition de la fonction B'(T):

In [10]:
def n_prime(gamma_maj,gamma_sh,n0):
    return n0*(gamma_chapeau_p()*gamma_maj(gamma_sh) + 1)/(gamma_chapeau_p() - 1)


def B_prime(epsilonB, gamma_sh,n0):
    return (np.sqrt(8*const.pi*epsilonB*e_i_p(gamma_maj,gamma_chapeau_p, gamma_sh,n0)))




In [9]:
def nu_prime_m(epsilon_B, gamma_sh,n0 ):
    return 3/16 * gamma_prime_m()**2 * const.q_e*B_prime(epsilon_B,gamma_maj,gamma_chapeau_p, gamma_sh,n0) / (const.m_e * const.celer)

def nu_prime_c(n0,epsilon_B, t, gamma_sh):
    return 3/16 * gamma_prime_c(n0,epsilon_B,gamma_sh, t)**2 * const.q_e *B_prime(epsilon_B, gamma_sh, n0) / (const.m_e * const.celer)

Pour la fonction d'émission qui est définie dans le papier par les équations (24) et (25) la renormalisation nous donne:

$$ x = \frac{\nu'}{\nu_{m}'}$$
Pour déterminer le régime de refroidissement on définit un rapport 
$$ \kappa = \frac{\nu_{c}'(\epsilon_{B},t)}{\nu_{m}'(\epsilon_{B})}$$
on obtient donc:
Si $\kappa > 1$:

 \[ $$ \frac{\epsilon_{\nu'}'}{\epsilon_{\nu', p}'} =$$
\begin{cases}
\text{$x^{\frac{1}{3}}$  pour  $x <1$} \\
\text{$x^{-(p-1)/2}$ pour  $1\leq x \leq \kappa$} \\
\text{$\kappa^{-1/2} x^{-p/2}$ pour  $\kappa < x$}
\end{cases}
\]

Si $\kappa \leq 1$:

 \[ $$ \frac{\epsilon_{\nu'}'}{\epsilon_{\nu', p}'} =$$
\begin{cases}
\text{$x^{\frac{1}{3}} \kappa^{-1/3}$  pour  $x <\kappa$} \\
\text{$\kappa^{1/2} x^{-1/2}$ pour  $\kappa\leq x < 1$} \\
\text{$\kappa^{-1/2} x^{-p/2}$ pour  $1  \leq x$}
\end{cases}
\]

In [13]:
#Pour la fonction d'émission on renormalise tout:
epsi_b_gaussianJ = 2.44*(1/np.power(10,4))
epsi_b_hollow_coneJ = 4.12*(1/np.power(10,5))
epsi_b_SpindleJ = 6.60*(1/np.power(10,4))
#pour l'instant en première approximation on prend p = 2.05


def power_I(x,y):
    return 1/np.power(x,y)

#Rapport de comparaison pour les refroidissemts entre nu_prim_c et nu_prim_m
def Kappa(n0,epsilon_B,t,gamma_sh):
    return nu_prime_c(n0, epsilon_B, gamma_sh,t)/nu_prime_m(epsilon_B, gamma_sh, n0)


In [100]:
def FastcEmission(n0,x, Kappa,p,epsilon_B,t,gamma_sh):
    if x > Kappa(n0,epsilon_B,t,gamma_sh):
        return np.power(Kappa(n0,epsilon_B,t,gamma_sh),-1/3)*np.power(x,1/3)
    elif x >= Kappa(n0,epsilon_B,t,gamma_sh) and x <1:
        return np.power(Kappa(n0,epsilon_B,t,gamma_sh),1/2)*power_I(x,-1/2)
    else:
        power_I(Kappa(n0,epsilon_B,t,gamma_sh),-1/2)*power_I(x,-p/2)

FastcEmission_vect = np.vectorize(lambda x: FastcEmission(x, Kappa,p,epsi_b_gaussianJ,n0,t,gamma_sh))

TypeError: 'float' object is not callable

In [95]:



#Cas du slow-cooling
def SlowcEmission(x,Kappa,p,epsilon_B,t,gamma_sh,n0):
    if x < 1:
        return np.power(x,1/3)
    elif x>=1 and x < Kappa(n0,epsilon_B,t,gamma_sh):
        return power_I(x,-(p-1)/2)
    else:
        return power_I(Kappa(n0,epsilon_B,t,gamma_sh),-1/2)*power_I(-p/2)

T1 = 10**14

#Vectorisation des fonctions liées aux régimes de refroidissement
SlowcEmission_vect = np.vectorize(lambda x: SlowcEmission(x,Kappa,p,epsi_b_gaussianJ,n0,t,T1))
FastcEmission_vect = np.vectorize(lambda x: FastcEmission(x, Kappa,p,epsi_b_gaussianJ,n0,t,T1))


#Fonction d'émission
def Emission_function(n0,Kappa,x,epsilon_B,t,gamma_sh):
    if Kappa(n0,epsilon_B,t,gamma_sh) > 1:
        return SlowcEmission_vect(x)
    else:
        return FastcEmission_vect(x)

    



In [97]:
x = np.linspace(0,10,5000)
y_H = Emission_function(Kappa,x,p,epsi_b_hollow_coneJ,t,T1)
plt.plot(x,y_H)
plt.title("Emissivity function for hollow-cone jet")
plt.xlabel("x")
plt.ylabel("$\frac{\epsilon_{\nu'}'}{\epsilon_{\nu', p}'}")
plt.show()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Groupe équation 31